## How BLE works

http://www.blesstags.eu/2018/08/services-characteristics-descriptors.html

A BLE device has services, characteristics, and descriptors in the following tree structure:

```{mermaid}
    graph LR 
    A[BLE Device] --> S1[Service]
    A --> S2[Service]

    S1 --> C1[Characteristic]
    C1 --> D1[Descriptor]
    C1 --> D2[Descriptor]


    S1 --> C2[Characteristic]

    S2 --> C3[Characteristic]
    C3 --> D3[Descriptor]

    S2 --> C4[Characteristic]

```

The behavior of a BLE device is given by the different **Services** that encapsulate different parts of this behavior

Mainly, a service is a container for different data items – these data items are called are called **Characteristics**.

A **Characteristic** is a data item that can be either:
* __read__ (e.g. a sensor value, battery level, etc.) 
* or __written__ (e.g. a configuration value, etc.). 

**Descriptor** - ?

In [ ]:
# | default_exp bt

## Aranet4 BLE services

In [ ]:
# | export

from dataclasses import dataclass, field
from datetime import datetime, timedelta
from typing import Any, Dict, List, Union

import aranet4
import pandas as pd
import rich


@dataclass
class AranetDevice:
    address: str
    name: str

    def get_current_readings(self) -> aranet4.client.CurrentReading:
        """fetch the current readings from the device"""
        return aranet4.client.get_current_readings(self.address)

    def get_history(
        self, entry_filter: Dict, as_df: bool = True
    ) -> Union[pd.DataFrame, List[aranet4.client.RecordItem]]:
        """fetch the history from the device"""
        records = aranet4.client.get_all_records(
            self.address,
            entry_filter,
            remove_empty=True,  # This will remove blank records, if range parameters (start,end,last) are specified
        )

        if as_df:
            return pd.DataFrame(records.value)
        else:
            return records


class AranetScanner:
    def __init__(self):
        self.devices: Dict[str, AranetDevice] = dict()

    def on_scan(self, advertisement: Any) -> None:
        device = AranetDevice(
            address=advertisement.device.address,
            name=advertisement.device.name,
        )

        self.devices[device.address] = device

    def find_nearby(self, duration: int = 5):
        print("Scanning Aranet4 devices...")
        aranet4.client.find_nearby(self.on_scan, duration)
        print(f"\nFound {len(self.devices)} devices:\n")

In [ ]:
# | export
import fastcore.foundation

if not fastcore.foundation.in_ipython():
    scanner = AranetScanner()
    scanner.find_nearby()

    for _, device in scanner.devices.items():
        # rich.inspect(device)
        rich.inspect(device.get_current_readings())
        records = device.get_history(
            entry_filter={"start": datetime.now() - timedelta(minutes=30)}, as_df=True
        )

        print(f"downloaded {len(records)} records for {device.name} ({device.address})")

**Publishing Aranet4 data to mqtt**

```{mermaid}
    graph LR 
    PY[Python] --> MQTT[MQTT]
    MQTT --> HA[Home Assistant]
    MQTT --> G[Grafana]
```


In [ ]:
# | hide
# | notest

# import sys
# from dataclasses import asdict

# import paho.mqtt.publish as publish


# def buildMsgs(readings, topic):
#     return [
#         (topic + "temperature", readings["temperature"]),
#         (topic + "pressure", readings["pressure"]),
#         (topic + "humidity", readings["humidity"]),
#         (topic + "co2", readings["co2"]),
#         (topic + "battery", readings["battery"]),
#     ]


# def readArg(argv, key, default, error="Invalid value"):
#     if key in argv:
#         idx = argv.index(key) + 1
#         if idx >= len(argv):
#             print(error)
#             raise Exception(error)
#         return argv[idx]
#     return default


# def main(argv):
#     if len(argv) < 3:
#         print("Missing device address, topic base and/or hostname.")
#         argv[0] = "?"

#     if "help" in argv or "?" in argv:
#         print("Usage: python publish.py DEVICE_ADDRESS HOSTNAME TOPIC_BASE [OPTIONS]")
#         print("  -P  <port>      Broker port")
#         print("  -u  <user>      Auth user name")
#         print("  -p  <password>  Auth user password")

#         print("")
#         return

#     device_mac = argv[0]
#     host = argv[1]
#     topic = argv[2]

#     port = readArg(argv, "-P", "1883")
#     user = readArg(argv, "-u", "")
#     pwd = readArg(argv, "-p", "")

#     auth = None

#     if len(user) > 0:
#         auth = {"username": user}
#         if len(pwd) > 0:
#             auth["password"] = pwd

#     if topic[-1] != "/":
#         topic += "/"

#     current = aranet4.client.get_current_readings(device_mac)

#     print("Publishing results...")
#     publish.multiple(
#         buildMsgs(asdict(current), topic), hostname=host, port=int(port), auth=auth
#     )